#### Initialize the Spark 

In [11]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Bronze") \
    .config("spark.jars.packages", "com.crealytics:spark-excel_2.12:0.13.5") \
    .getOrCreate()

#### Schemas for files

In [12]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

# Define the schema for the Locations table
locations_Schema = StructType([
    StructField("LocationId", StringType(), True),
    StructField("OrganisationID", StringType(), True),
    StructField("LocationPostCode", StringType(), True)
])

# Define the schema for the NSPL21_AUG_2023_UK table
NSPL21_AUG_2023_UK_Schema = StructType([
    StructField("pcd", StringType(), True),
    StructField("pcd2", StringType(), True),
    StructField("pcds", StringType(), True),
    StructField("dointr", IntegerType(), True),
    StructField("doterm", IntegerType(), True),
    StructField("usertype", IntegerType(), True),
    StructField("oseast1m", IntegerType(), True),
    StructField("osnrth1m", IntegerType(), True),
    StructField("osgrdind", IntegerType(), True),
    StructField("oa21", StringType(), True),
    StructField("cty", StringType(), True),
    StructField("ced", StringType(), True),
    StructField("laua", StringType(), True),
    StructField("ward", StringType(), True),
    StructField("nhser", StringType(), True),
    StructField("ctry", StringType(), True),
    StructField("rgn", StringType(), True),
    StructField("pcon", StringType(), True),
    StructField("ttwa", StringType(), True),
    StructField("itl", StringType(), True),
    StructField("npark", StringType(), True),
    StructField("lsoa21", StringType(), True),
    StructField("msoa21", StringType(), True),
    StructField("wz11", StringType(), True),
    StructField("sicbl", StringType(), True),
    StructField("bua22", StringType(), True),
    StructField("ru11ind", StringType(), True),
    StructField("oac11", StringType(), True),
    StructField("lat", DoubleType(), True),
    StructField("long", DoubleType(), True),
    StructField("lep1", StringType(), True),
    StructField("lep2", StringType(), True),
    StructField("pfa", StringType(), True),
    StructField("imd", IntegerType(), True),
    StructField("icb", StringType(), True)
])

NSP21CL_AUG23_UK_LU_Schema = StructType([
    StructField("pcd7", StringType(), True),
    StructField("pcd8", StringType(), True),
    StructField("pcds", StringType(), True),
    StructField("dointr", IntegerType(), True),
    StructField("doterm", IntegerType(), True),
    StructField("usertype", IntegerType(), True),
    StructField("oseast1m", IntegerType(), True),
    StructField("osnrth1m", IntegerType(), True),
    StructField("oa21cd", StringType(), True),
    StructField("oac11cd", StringType(), True),
    StructField("oac11nm", StringType(), True),
    StructField("wz11cd", StringType(), True),
    StructField("wzc11cd", StringType(), True),
    StructField("wzc11nm", StringType(), True),
    StructField("lsoa21cd", StringType(), True),
    StructField("lsoa21nm", StringType(), True),
    StructField("msoa21cd", StringType(), True),
    StructField("msoa21nm", StringType(), True),
    StructField("soac11cd", StringType(), True),
    StructField("soac11nm", StringType(), True),
    StructField("ladcd", StringType(), True),
    StructField("ladnm", StringType(), True),
    StructField("ladnmw", StringType(), True),
    StructField("laccd", StringType(), True),
    StructField("lacnm", StringType(), True)
])


#### Read the Files

In [13]:
# Read the Locations
df_locations = spark.read \
    .format("com.crealytics.spark.excel") \
    .option("header", "true") \
    .schema(locations_Schema) \
    .load("Bronze/locations.xlsx")

df_locations.printSchema()
df_locations.show(5,truncate=False)

# Read the NSPL21_AUG_2023_UK
df_lookup_full = spark.read \
    .option("header", True) \
    .schema(NSPL21_AUG_2023_UK_Schema) \
    .csv("Bronze/NSPL21_AUG_2023_UK.csv")
    
df_lookup_full.printSchema()
df_lookup_full.show(5,truncate=False)

# Read the NSP21CL_AUG23_UK_LU
df_rural_lookup_full_CL = spark.read \
    .option("header", True) \
    .schema(NSP21CL_AUG23_UK_LU_Schema) \
    .csv("Bronze/NSP21CL_AUG23_UK_LU.csv")
    
df_rural_lookup_full_CL.printSchema()
df_rural_lookup_full_CL.show(5,truncate=False)

root
 |-- LocationId: string (nullable = true)
 |-- OrganisationID: string (nullable = true)
 |-- LocationPostCode: string (nullable = true)

+------------------------------------+------------------------------------+----------------+
|LocationId                          |OrganisationID                      |LocationPostCode|
+------------------------------------+------------------------------------+----------------+
|d2587969-bb36-4f68-9718-1de5f95b3316|7e26597f-e458-4ced-a922-49fc04b17bc8|BB2 6PS         |
|bb6febc5-b986-4a0b-b537-03a56fa321a4|b69d7b9d-998e-4be2-a7d9-3c2809b8363b|LE11 2ER        |
|ce0f066c-99dc-4fe2-aedd-4a61ffcf0c15|4aacc297-5d86-4032-9b42-09ad0575b2eb|H91 WR26        |
|2a602313-0008-4b40-a345-207a553aafec|16cd29d4-e34a-49bd-b5bd-266b56a4f578|SA1 7LQ         |
|ff7ba5dd-dfba-4f31-a1bc-335e3385e543|3eac8971-cdd4-4fd9-bc15-0f24cbaa439b|GL5 4JE         |
+------------------------------------+------------------------------------+----------------+
only showing top 5 ro

##### Count nulls and duplicates for each column in files

In [14]:
from pyspark.sql.functions import col, count, when

# Locations
null_counts = df_locations.select([count(when(col(c).isNull(), c)).alias(c) for c in df_locations.columns])
print("Number of nulls for each column:")
null_counts.show()

for c in df_locations.columns:
    duplicate_count = df_locations.groupBy(c).count().filter("`count` > 1").count()
    print(f"Number of duplicates in {c}: {duplicate_count}")
    
# NSPL21_AUG_2023_UK
df_lookup_full_null_counts = df_lookup_full.select([count(when(col(c).isNull(), c)).alias(c) for c in df_lookup_full.columns])
print("Number of nulls for each column:")
df_lookup_full_null_counts.show()
print("Number of duplicates for each column:")

duplicate_counts = {}

for c in df_lookup_full.columns:
    duplicates = df_lookup_full.groupBy(c).count().filter("`count` > 1").count()
    duplicate_counts[c] = duplicates

df_lookup_full_duplicate_counts = spark.createDataFrame(duplicate_counts.items(), schema=["Column", "DuplicateCount"])
df_lookup_full_duplicate_counts.show(n=30,truncate=False)

# NSP21CL_AUG23_UK_LU
df_rural_lookup_full_CL_null_counts = df_rural_lookup_full_CL.select([count(when(col(c).isNull(), c)).alias(c) for c in df_rural_lookup_full_CL.columns])
print("Number of nulls for each column:")
df_rural_lookup_full_CL_null_counts.show()
print("Number of duplicates for each column:")

duplicate_counts = {}

for c in df_rural_lookup_full_CL.columns:
    duplicates_rural_lookup_full_CL = df_rural_lookup_full_CL.groupBy(c).count().filter("`count` > 1").count()
    duplicate_counts[c] = duplicates_rural_lookup_full_CL

df_duplicate_counts_rural_lookup_full_CL = spark.createDataFrame(duplicate_counts.items(), schema=["Column", "DuplicateCount"])
df_duplicate_counts_rural_lookup_full_CL.show(truncate=False,n=30)

Number of nulls for each column:
+----------+--------------+----------------+
|LocationId|OrganisationID|LocationPostCode|
+----------+--------------+----------------+
|         0|             0|              72|
+----------+--------------+----------------+

Number of duplicates in LocationId: 0
Number of duplicates in OrganisationID: 241
Number of duplicates in LocationPostCode: 40
Number of nulls for each column:


+---+----+----+------+-------+--------+--------+--------+--------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+-----+-----+-----+-------+-----+---+----+-----+-------+-----+---+-----+
|pcd|pcd2|pcds|dointr| doterm|usertype|oseast1m|osnrth1m|osgrdind| oa21|  cty|  ced| laua| ward|nhser| ctry|  rgn| pcon| ttwa|  itl|npark|lsoa21|msoa21| wz11|sicbl|bua22|ru11ind|oac11|lat|long| lep1|   lep2|  pfa|imd|  icb|
+---+----+----+------+-------+--------+--------+--------+--------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+-----+-----+-----+-------+-----+---+----+-----+-------+-----+---+-----+
|  0|   0|   0|     0|1796370|       0|   23901|   23901|       0|10805|10805|10805|10805|10805|10805|10799|10805|10805|10799|10805| 8083| 10805| 10799|10799|10805|10799|  70752|10799|  0|   0|10805|2656704|10805|  0|10805|
+---+----+----+------+-------+--------+--------+--------+--------+-----+-----+-----+-----+-----+-----+--

+--------+--------------+
|Column  |DuplicateCount|
+--------+--------------+
|pcd     |0             |
|pcd2    |0             |
|pcds    |0             |
|dointr  |538           |
|doterm  |415           |
|usertype|2             |
|oseast1m|347128        |
|osnrth1m|397648        |
|osgrdind|8             |
|oa21    |230508        |
|cty     |39            |
|ced     |1579          |
|laua    |377           |
|ward    |8469          |
|nhser   |13            |
|ctry    |7             |
|rgn     |15            |
|pcon    |653           |
|ttwa    |231           |
|itl     |391           |
|npark   |22            |
|lsoa21  |43501         |
|msoa21  |8547          |
|wz11    |60707         |
|sicbl   |155           |
|bua22   |4171          |
|ru11ind |20            |
|oac11   |78            |
|lat     |415937        |
|long    |382086        |
+--------+--------------+
only showing top 30 rows

Number of nulls for each column:


+----+----+----+------+-------+--------+--------+--------+------+-------+-------+------+-------+-------+--------+--------+--------+--------+--------+--------+-----+-----+-------+------+------+
|pcd7|pcd8|pcds|dointr| doterm|usertype|oseast1m|osnrth1m|oa21cd|oac11cd|oac11nm|wz11cd|wzc11cd|wzc11nm|lsoa21cd|lsoa21nm|msoa21cd|msoa21nm|soac11cd|soac11nm|ladcd|ladnm| ladnmw| laccd| lacnm|
+----+----+----+------+-------+--------+--------+--------+------+-------+-------+------+-------+-------+--------+--------+--------+--------+--------+--------+-----+-----+-------+------+------+
|   0|   0|   0|     0|1796370|       0|   23901|   23901| 10805|  10799|  10799| 10799|  10799|  10799|   10805|  312978|   10799|  312238|  230421|  230421|10805|10805|2553918|119222|119222|
+----+----+----+------+-------+--------+--------+--------+------+-------+-------+------+-------+-------+--------+--------+--------+--------+--------+--------+-----+-----+-------+------+------+

Number of duplicates for each colu

+--------+--------------+
|Column  |DuplicateCount|
+--------+--------------+
|pcd7    |0             |
|pcd8    |0             |
|pcds    |0             |
|dointr  |538           |
|doterm  |415           |
|usertype|2             |
|oseast1m|347128        |
|osnrth1m|397648        |
|oa21cd  |230508        |
|oac11cd |78            |
|oac11nm |78            |
|wz11cd  |60707         |
|wzc11cd |31            |
|wzc11nm |31            |
|lsoa21cd|43501         |
|lsoa21nm|35669         |
|msoa21cd|8547          |
|msoa21nm|7264          |
|soac11cd|26            |
|soac11nm|26            |
|ladcd   |377           |
|ladnm   |377           |
|ladnmw  |23            |
|laccd   |26            |
|lacnm   |26            |
+--------+--------------+



#### Add NK (Not Known) for null values in the files where necessary

In [15]:
# Add NK (Not Known) for null values in LocationPostCode column 
df_locations_NK = df_locations.fillna("NK", subset=["LocationPostCode"])
df_locations_NK.show(5)

df_filtered_LocationPostCode_NK = df_locations.filter(df_locations["LocationPostCode"] == "NK")
df_filtered_LocationPostCode_NK.show(5)

# Add NK (Not Known) for null values in ladnm column 
df_rural_lookup_full_CL_NK = df_rural_lookup_full_CL.fillna("NK", subset=["ladnm"])
df_rural_lookup_full_CL_NK.show(5)

df_filtered_ladnm = df_rural_lookup_full_CL.filter(df_rural_lookup_full_CL["ladnm"] == "NK")
df_filtered_ladnm.show(5)

# Add NK (Not Known) for null values in ru11ind column 
df_lookup_full_NK = df_lookup_full.fillna("NK", subset=["ru11ind"])
df_lookup_full_NK.show(5)

df_filtered_ru11ind_NK = df_lookup_full.filter(df_lookup_full["ru11ind"] == "NK")
df_filtered_ru11ind_NK.show(5)


+--------------------+--------------------+----------------+
|          LocationId|      OrganisationID|LocationPostCode|
+--------------------+--------------------+----------------+
|d2587969-bb36-4f6...|7e26597f-e458-4ce...|         BB2 6PS|
|bb6febc5-b986-4a0...|b69d7b9d-998e-4be...|        LE11 2ER|
|ce0f066c-99dc-4fe...|4aacc297-5d86-403...|        H91 WR26|
|2a602313-0008-4b4...|16cd29d4-e34a-49b...|         SA1 7LQ|
|ff7ba5dd-dfba-4f3...|3eac8971-cdd4-4fd...|         GL5 4JE|
+--------------------+--------------------+----------------+
only showing top 5 rows

+----------+--------------+----------------+
|LocationId|OrganisationID|LocationPostCode|
+----------+--------------+----------------+
+----------+--------------+----------------+

+-------+--------+-------+------+------+--------+--------+--------+---------+-------+--------------------+---------+-------+--------------------+---------+--------+---------+--------+--------+--------------------+---------+-------------+------+-

+----+----+----+------+------+--------+--------+--------+------+-------+-------+------+-------+-------+--------+--------+--------+--------+--------+--------+-----+-----+------+-----+-----+
|pcd7|pcd8|pcds|dointr|doterm|usertype|oseast1m|osnrth1m|oa21cd|oac11cd|oac11nm|wz11cd|wzc11cd|wzc11nm|lsoa21cd|lsoa21nm|msoa21cd|msoa21nm|soac11cd|soac11nm|ladcd|ladnm|ladnmw|laccd|lacnm|
+----+----+----+------+------+--------+--------+--------+------+-------+-------+------+-------+-------+--------+--------+--------+--------+--------+--------+-----+-----+------+-----+-----+
+----+----+----+------+------+--------+--------+--------+------+-------+-------+------+-------+-------+--------+--------+--------+--------+--------+--------+-----+-----+------+-----+-----+

+-------+--------+-------+------+------+--------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+-------

+---+----+----+------+------+--------+--------+--------+--------+----+---+---+----+----+-----+----+---+----+----+---+-----+------+------+----+-----+-----+-------+-----+---+----+----+----+---+---+---+
|pcd|pcd2|pcds|dointr|doterm|usertype|oseast1m|osnrth1m|osgrdind|oa21|cty|ced|laua|ward|nhser|ctry|rgn|pcon|ttwa|itl|npark|lsoa21|msoa21|wz11|sicbl|bua22|ru11ind|oac11|lat|long|lep1|lep2|pfa|imd|icb|
+---+----+----+------+------+--------+--------+--------+--------+----+---+---+----+----+-----+----+---+----+----+---+-----+------+------+----+-----+-----+-------+-----+---+----+----+----+---+---+---+
+---+----+----+------+------+--------+--------+--------+--------+----+---+---+----+----+-----+----+---+----+----+---+-----+------+------+----+-----+-----+-------+-----+---+----+----+----+---+---+---+



#### Clean the columns have the post codes

In [16]:
from pyspark.sql.functions import col, regexp_replace, trim, length, when

# Cleaning and flagging for df_locations_NK
df_locations_cleaned = df_locations_NK.withColumn("LocationPostCode_clean",
                                                  regexp_replace(col("LocationPostCode"), "\\s+", "")) \
                                       .withColumn("LocationPostCode_clean",
                                                   regexp_replace(col("LocationPostCode_clean"), "[\\p{Punct}]", "")) \
                                       .withColumn("LocationPostCode_clean",
                                                   trim(col("LocationPostCode_clean"))) \
                                       .withColumn("valid_LocationPostCode",
                                                   when((length(col("LocationPostCode_clean")) >= 5) & 
                                                        (length(col("LocationPostCode_clean")) <= 8), True)
                                                   .otherwise(False))

df_locations_cleaned.show(n=1600,truncate=False)
df_locations_cleaned.filter(col("valid_LocationPostCode") == False).show(truncate=False)

# Cleaning and flagging for df_rural_lookup_full_CL_NK
df_rural_lookup_full_CL_cleaned = df_rural_lookup_full_CL_NK.withColumn("pcds_clean",
                                                                        regexp_replace(col("pcds"), "\\s+", "")) \
                                                             .withColumn("pcds_clean",
                                                                         regexp_replace(col("pcds_clean"), "[\\p{Punct}]", "")) \
                                                             .withColumn("pcds_clean",
                                                                         trim(col("pcds_clean"))) \
                                                             .withColumn("valid_pcds",
                                                                         when((length(col("pcds_clean")) >= 5) & 
                                                                              (length(col("pcds_clean")) <= 8), True)
                                                                         .otherwise(False))

df_rural_lookup_full_CL_cleaned.show(5,truncate=False)
df_rural_lookup_full_CL_cleaned.filter(col("valid_pcds") == False).show(truncate=False)

# Cleaning and flagging for df_lookup_full_NK
df_lookup_full_CL_cleaned = df_lookup_full_NK.withColumn("pcds_clean",
                                                         regexp_replace(col("pcds"), "\\s+", "")) \
                                              .withColumn("pcds_clean",
                                                          regexp_replace(col("pcds_clean"), "[\\p{Punct}]", "")) \
                                              .withColumn("pcds_clean",
                                                          trim(col("pcds_clean"))) \
                                              .withColumn("valid_pcds",
                                                          when((length(col("pcds_clean")) >= 5) & 
                                                               (length(col("pcds_clean")) <= 8), True)
                                                          .otherwise(False))

df_lookup_full_CL_cleaned.show(5,truncate=False)
df_lookup_full_CL_cleaned.filter(col("valid_pcds") == False).show()


+------------------------------------+------------------------------------+----------------+----------------------+----------------------+
|LocationId                          |OrganisationID                      |LocationPostCode|LocationPostCode_clean|valid_LocationPostCode|
+------------------------------------+------------------------------------+----------------+----------------------+----------------------+
|d2587969-bb36-4f68-9718-1de5f95b3316|7e26597f-e458-4ced-a922-49fc04b17bc8|BB2 6PS         |BB26PS                |true                  |
|bb6febc5-b986-4a0b-b537-03a56fa321a4|b69d7b9d-998e-4be2-a7d9-3c2809b8363b|LE11 2ER        |LE112ER               |true                  |
|ce0f066c-99dc-4fe2-aedd-4a61ffcf0c15|4aacc297-5d86-4032-9b42-09ad0575b2eb|H91 WR26        |H91WR26               |true                  |
|2a602313-0008-4b40-a345-207a553aafec|16cd29d4-e34a-49bd-b5bd-266b56a4f578|SA1 7LQ         |SA17LQ                |true                  |
|ff7ba5dd-dfba-4f31-a1bc-33

+----+----+----+------+------+--------+--------+--------+------+-------+-------+------+-------+-------+--------+--------+--------+--------+--------+--------+-----+-----+------+-----+-----+----------+----------+
|pcd7|pcd8|pcds|dointr|doterm|usertype|oseast1m|osnrth1m|oa21cd|oac11cd|oac11nm|wz11cd|wzc11cd|wzc11nm|lsoa21cd|lsoa21nm|msoa21cd|msoa21nm|soac11cd|soac11nm|ladcd|ladnm|ladnmw|laccd|lacnm|pcds_clean|valid_pcds|
+----+----+----+------+------+--------+--------+--------+------+-------+-------+------+-------+-------+--------+--------+--------+--------+--------+--------+-----+-----+------+-----+-----+----------+----------+
+----+----+----+------+------+--------+--------+--------+------+-------+-------+------+-------+-------+--------+--------+--------+--------+--------+--------+-----+-----+------+-----+-----+----------+----------+

+-------+--------+-------+------+------+--------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------

+---+----+----+------+------+--------+--------+--------+--------+----+---+---+----+----+-----+----+---+----+----+---+-----+------+------+----+-----+-----+-------+-----+---+----+----+----+---+---+---+----------+----------+
|pcd|pcd2|pcds|dointr|doterm|usertype|oseast1m|osnrth1m|osgrdind|oa21|cty|ced|laua|ward|nhser|ctry|rgn|pcon|ttwa|itl|npark|lsoa21|msoa21|wz11|sicbl|bua22|ru11ind|oac11|lat|long|lep1|lep2|pfa|imd|icb|pcds_clean|valid_pcds|
+---+----+----+------+------+--------+--------+--------+--------+----+---+---+----+----+-----+----+---+----+----+---+-----+------+------+----+-----+-----+-------+-----+---+----+----+----+---+---+---+----------+----------+
+---+----+----+------+------+--------+--------+--------+--------+----+---+---+----+----+-----+----+---+----+----+---+-----+------+------+----+-----+-----+-------+-----+---+----+----+----+---+---+---+----------+----------+



#### Save the tables to the Silver

In [17]:
df_locations_cleaned.select("LocationId", "OrganisationID", "LocationPostCode_clean") \
.write.csv("Silver/locations_cleaned.csv", header=True, mode="overwrite")

df_lookup_full_CL_cleaned.select("pcds_clean", "ru11ind", "lat", "long") \
    .write.csv("Silver/lookup_full_cleaned.csv", header=True, mode="overwrite")
    
df_rural_lookup_full_CL_cleaned.select("pcds_clean", "ladnm") \
    .write.csv("Silver/rural_lookup_full_CL_cleaned.csv", header=True, mode="overwrite")

print("The tables have been saved to the Silver.")


The tables have been saved to the Silver.


#### Check pcd, pcd2, and pcds columns whether they have same data or not

In [18]:
# from pyspark.sql.functions import col, regexp_replace

# # Remove spaces from all three columns
# df = df_lookup_full.withColumn("pcd_clean", regexp_replace(col("pcd"), "\\s+", ""))
# df = df.withColumn("pcd2_clean", regexp_replace(col("pcd2"), "\\s+", ""))
# df = df.withColumn("pcds_clean", regexp_replace(col("pcds"), "\\s+", ""))
# df_lookup_full_filtered = df
# df_lookup_full_filtered.show()

# # Now compare the cleaned columns
# unmatched_rows = df_lookup_full_filtered.filter(
#     (col("pcd_clean") != col("pcd2_clean")) | 
#     (col("pcd_clean") != col("pcds_clean")) | 
#     (col("pcd2_clean") != col("pcds_clean"))
# )

# print("Number of unmatched rows: ",unmatched_rows.count())
# unmatched_rows.show(5)


#### Option to filter null values for files where necessary

In [19]:
# # Locations file
# print("Total rows: ",df_locations.count())
# df_locations_filtered = df_locations.filter(df_locations.LocationPostCode.isNotNull())
# print("Number of rows which are not null on LocationPostCode column:",df_locations_filtered.count())

# # NSPL21_AUG_2023_UK file
# print("Total rows: ",df_lookup_full.count())
# df_lookup_full_filtered = df_lookup_full.filter(df_lookup_full.ru11ind.isNotNull())
# print("Number of rows which are not null on ru11ind column:",df_lookup_full_filtered.count())
# print("Difference in rows:", df_lookup_full.count() - df_lookup_full_filtered.count())

# # NSP21CL_AUG23_UK_LU file
# total_rows = df_rural_lookup_full_CL.count()
# print("Total rows:", total_rows)

# df_rural_lookup_full_CL_filtered = df_rural_lookup_full_CL \
#     .filter(df_rural_lookup_full_CL.ladnm.isNotNull())

# filtered_rows = df_rural_lookup_full_CL_filtered.count()
# print("Rows after removing nulls on ladnm columns:", filtered_rows)
# print("Difference in rows:", total_rows - filtered_rows)


#### Option to fill null values in 'ru11ind' column

In [20]:
# from pyspark.sql.functions import min, max

# # Group by 'ru11ind' and calculate min/max of 'lat' and 'long'
# agg_df = df_lookup_full.groupBy("ru11ind").agg(
#     min("lat").alias("min_lat"),
#     max("lat").alias("max_lat"),
#     min("long").alias("min_long"),
#     max("long").alias("max_long")
# ).orderBy("ru11ind")

# # Show the result
# agg_df.show()
